In [1]:
import sys
sys.path.append('../src/')

In [2]:
# TODO: Add code definitions here
import json
import numpy as np
import time
import matplotlib.pyplot as plt
import torch
from learning import NetworkBGP
%matplotlib inline

## Opening Data

In [3]:
with open('../data/extracted_one_min_nov15.json') as f:
    full = json.load(f)
full[:3]

[{'time': 1542313320,
  'composite': {'prefix': '2620:0:2f0::', 'mask': 48, 'dest': 32629},
  'full_path': [63956, 4637, 174, 32629]},
 {'time': 1542313320,
  'composite': {'prefix': '2a0b:d080::', 'mask': 32, 'dest': 205747},
  'full_path': [63956, 13030, 60804, 174, 6830, 205747]},
 {'time': 1542313320,
  'composite': {'prefix': '94.185.102.0', 'mask': 24, 'dest': 198013},
  'full_path': [2914, 3356, 3549, 5392, 5392, 198013]}]

## Creating a Network

In [4]:
net_manager = NetworkBGP(full, 100)

There are 8768 messages to be labeled
There are 2191 messages to be labeled


In [ ]:
net_manager.Xtrain[:3]

tensor([[-1.6611,  2.5951,  1.3881, -0.4347],
        [-1.6611,  2.5996,  0.3207,  1.4575],
        [-1.6611, -0.3946, -0.2130,  1.3730]], dtype=torch.float64)

In [ ]:
net_manager.net

DistinctNN(
  (h1): Linear(in_features=4, out_features=100, bias=True)
  (a1): ReLU()
  (h2): Linear(in_features=100, out_features=100, bias=True)
  (a2): ReLU()
  (raw_out): Linear(in_features=100, out_features=2, bias=True)
  (out_act): LogSoftmax()
)

## Training the Network

In [ ]:
start = time.time()
losses = net_manager.train_network(num_iterations=200)
print('Training took {}'.format(time.time() - start))

In [ ]:
losses

In [ ]:
plt.plot(losses)
plt.ylabel('Average Error (loss)')
plt.xlabel('Training iteration')
plt.title('Errors During Training')
plt.show()

In [ ]:
losses[-1]

## Testing the Network
### Data used for Training

In [ ]:
alpha=1

In [ ]:
predicted = torch.max(net_manager.net(net_manager.Xtrain), 1)[1].numpy()
actual = net_manager.Ttrain.numpy()

In [ ]:
predicted

In [ ]:
net_manager.net(net_manager.Xtrain)

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(10, 10)
ax.plot(actual, 'o', alpha=alpha, label='actual')
ax.plot(predicted, 'o', alpha=alpha, label='predicted')
plt.legend()
plt.ylabel('Class (1 = distinct, 0 = duplicate)')
plt.xlabel('Data point index, ordered by time')
plt.title('Classification for Training Set')
plt.show()

In [ ]:
actual_ones = np.where(actual.flatten() == 1)[0]
should_be_ones = np.take(predicted, actual_ones)
actual_ones = np.take(actual, actual_ones)
sum(should_be_ones == actual_ones) / len(actual_ones)

### On New Data

In [ ]:
predicted = torch.max(net_manager.net(net_manager.Xtest), 1)[1].numpy()
actual = net_manager.Ttest.numpy()
fig, ax = plt.subplots()
fig.set_size_inches(10, 10)
ax.plot(actual, 'o', alpha=alpha, label='actual')
ax.plot(predicted, 'o', alpha=alpha, label='predicted')
plt.legend()
plt.ylabel('Class (1 = distinct, 0 = duplicate)')
plt.xlabel('Data point index, ordered by time')
plt.title('Classification for Testing Set')
plt.show()

In [ ]:
np.sum(predicted == actual.flatten()) / len(predicted)

In [ ]:
actual_ones = np.where(actual.flatten() == 1)[0]
should_be_ones = np.take(predicted, actual_ones)
actual_ones = np.take(actual, actual_ones)
sum(should_be_ones == actual_ones) / len(actual_ones)